# =============================================
## GHI CHÚ TỔNG QUAN (NOTE – DÀNH CHO KAGGLE)
# =============================================
 **Mục tiêu**
 - Fine-tune mô hình T5 (seq2seq) cho bài toán Text-to-SQL trên Kaggle.
 - Dùng dataset công khai (b-mc2/sql-create-context, Clinton/Text-to-sql-v1, knowrohit07/know_sql),
 sau đó trộn, tiền xử lý (tokenize), huấn luyện, lưu mô hình.
 - Không thay đổi logic cốt lõi; chỉ bổ sung chú thích chi tiết.

 **Luồng chính**
 1) Cài đặt thư viện → Import.
 2) Nạp/ghép dataset có sẵn → lưu cache (merged_dataset).
 3) Tiền xử lý (tokenize) → lưu cache (tokenized_datasets).
 4) Kiểm thử zero-shot (baseline) trên model gốc.
 5) Huấn luyện full fine-tune → lưu mô hình đã fine-tune.
 6) Kiểm thử lại với mô hình fine-tuned + đánh giá (ROUGE) mẫu nhỏ.
 7) Đóng gói mô hình (zip) để tải xuống / dùng về sau.

 **Đầu ra (sản phẩm)**
 - Thư mục model đã fine-tune: /kaggle/working/sql_t5_finetuned
 - Có thể nạp về app (FastAPI/Flask/.NET-bridge) bằng from_pretrained("/kaggle/working/sql_t5_finetuned").

In [1]:
# Cài đặt thư viện datasets nếu chưa có
!pip install datasets



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Cài đặt thư viện evaluate
!pip install evaluate 



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# ==============================
# 2) IMPORT CÁC THƯ VIỆN CẦN THIẾT
# ==============================

In [5]:
pip uninstall -y tensorflow tensorflow-intel


Found existing installation: tensorflow 2.20.0Note: you may need to restart the kernel to use updated packages.

Uninstalling tensorflow-2.20.0:
  Successfully uninstalled tensorflow-2.20.0
Found existing installation: tensorflow-intel 2.17.0
Uninstalling tensorflow-intel-2.17.0:
  Successfully uninstalled tensorflow-intel-2.17.0


In [7]:
pip uninstall -y torch torchvision torchaudio


Found existing installation: torch 2.6.0+cpu
Uninstalling torch-2.6.0+cpu:
  Successfully uninstalled torch-2.6.0+cpu
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [8]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Note: you may need to restart the kernel to use updated packages.Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/109.2 MB ? eta -:--:--
   ---------------------------------------- 1.0/109.2 MB 6.3 MB/s eta 0:00:18
    --------------------------------------- 2.4/109.2 MB 6.4 MB/s eta 0:00:17
   - -------------------------------------- 3.7/109.2 MB 6.2 MB/s eta 0:00:17
   - -------------------------------------- 4.7/109.2 MB 5.9 MB/s eta 0:00:18
   -- ------------------------------------- 6.0/109.2 MB 6.0 MB/s eta 0:00:18
   -- ------------------------------------- 7.1/109.2 MB 6.0 MB/s eta 0:00:18
   --- ------------------------------------ 8.4/109.2 MB 6.0 MB/s eta 0:00:17
   --- ------------------------------------ 9.7/109.2 MB 6.0 MB/s eta 0:00:17
   ---- ----------------------------------- 11.0/109.2 MB 6.0 MB/s eta 0:00:17
   ---- ----------------------------------- 12.1/109.2 MB 6.0 MB/s eta 0:00:17
   ---- -----------------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch; 
print(torch.__version__, torch.cuda.is_available())

2.9.0+cpu False


In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk # Dataset Huggingface
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer # Model và Tokenizer cho trainning
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor
import time # Ghi thời gian trainning
import evaluate # Thư viện evaluate để đánh giá mô hình (BLEU, ROUGE)
import pandas as pd # Thư viện pandas để xử lý dữ liệu dạng bảng/kết quả
import numpy as np # Thư viện numpy để xử lý mảng


import warnings
warnings.filterwarnings("ignore") # Bỏ qua các cảnh báo không cần thiết

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# Import thư viện transformers và huggingface_hub
!pip install transformers huggingface_hub 


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer # Model và Tokenizer T5

import os # Thư viện os để thao tác với hệ thống tệp

# ==============================
# 3) CẤU HÌNH ĐƯỜNG DẪN VÀ THIẾT BỊ
# ==============================

In [7]:
drive_model_path = '/kaggle/working/' # Đường dẫn lưu model trên Kaggle

In [8]:
torch.cuda.is_available() # Kiểm tra GPU có sẵn không, trả về True hoặc False

False

In [ ]:
model_name='t5-small' # Tên model nền; code gốc chọn t5-small cho baseline/finetune

# Tạo tokenizer từ model nền
tokenizer = AutoTokenizer.from_pretrained(model_name) 

# Nạp model gốc (baseline) để so sánh trước/sau fine-tune
# Dùng dtype bfloat16 cho hiệu năng; chuyển model sang GPU nếu có
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to('cuda')

# ==============================
# 4) NẠP/CHUẨN BỊ DATASET (CÓ CACHE)
# ==============================

In [ ]:
# Cố gắng nạp dataset đã merge sẵn từ cache "merged_dataset" để tiết kiệm thời gian
try:
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    # Nếu không có cache, tiến hành tải và merge 3 dataset công khai và chia split thủ công
    
    # 4.1) b-mc2/sql-create-context (chứa question/context/answer)
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]') # 80% train
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]') # 10% test
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]') # 10% validation

    # 4.2) Clinton/Text-to-sql-v1 (đổi tên cột để thống nhất question/context/answer)

    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]') # 80% train
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text']) # Bỏ cột không cần thiết
    dataset_tts_train = dataset_tts_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'}) # Đổi tên cột cho thống nhất với dataset_scc
    
    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]') # 10% test
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text']) 
    dataset_tts_test  = dataset_tts_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'}) 
    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]') # 10% validation
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
    dataset_tts_val   = dataset_tts_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    # 4.3) knowrohit07/know_sql (đổi tên cột để thống nhất question/context/answer)
    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

    # 4.4) Ghép 3 nguồn lại bằng interleave_datasets để tăng đa dạng dữ liệu
    dataset = DatasetDict({ 'train': interleave_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train]),
                            'test': interleave_datasets([dataset_scc_test, dataset_tts_test, dataset_ks_test]),
                            'validation': interleave_datasets([dataset_scc_val, dataset_tts_val, dataset_ks_val])})

    # Lưu dataset đã merge vào cache "merged_dataset" để lần sau dùng lại
    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset")

dataset # Hiển thị thông tin dataset

README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

texttosqlv2.jsonl:   0%|          | 0.00/635M [00:00<?, ?B/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/118695 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14835 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14838 [00:00<?, ? examples/s]

Merged and Saved Dataset


DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14838
    })
})

In [ ]:
dataset['test'][0] # Kiểm tra một mẫu trong tập test

{'answer': 'SELECT date FROM table_name_11 WHERE away_team = "essendon"',
 'question': 'On what Date did the Away team essendon play?',
 'context': 'CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)'}

# ==============================
# 5) HÀM TIỀN XỬ LÝ (TOKENIZE) + TẠO PROMPT
# ==============================

Ở bước này, cần chuyển đổi các bộ dữ liệu thành dạng hướng dẫn rõ ràng cho mô hình ngôn ngữ lớn (LLM).

Sau đó, tiến hành tiền xử lý dữ liệu prompt-response bằng cách mã hóa (tokenize) để lấy ra các input_ids phục vụ cho quá trình huấn luyện.

Ghi chú: Chuyển dữ liệu dạng (context/question/answer) → (input_ids/labels) cho T5
 Prompt dạng:
 Tables:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n

In [ ]:
def tokenize_function(example): # Hàm tokenize_function để tiền xử lý dữ liệu

#     print(len(example["question"]))
# # Tiền tố/giữa/hậu cho prompt để mô hình rõ cấu trúc đầu vào
    start_prompt = "Tables:\n" # Bắt đầu với Tables:
    middle_prompt = "\n\nQuestion:\n" # Giữa là Question:
    end_prompt = "\n\nAnswer:\n" # Kết thúc với Answer:

    # Ghép promt theo từng cặp context/question
    data_zip = zip(example['context'], example['question']) # Ghép cặp context và question
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip] # Tạo prompt hoàn chỉnh
    
    # Mã hoá prompt → input_ids (padding/truncation theo mặc định max_length của tokenizer)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids # Tokenize prompt
    
    # Mã hoá câu trả lời (answer) → labels
    example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids # Tokenize answer
#     print(prompt[0])
#     print()

    return example

# Ghi chú: Hàm trên sẽ được áp vào cả 3 split (train/validation/test) bằng map(batched=True)
# batched=True để xử lý theo lô, tăng tốc độ xử lý
# Hàm tokenize_function xử lý tất cả dữ liệu trên tất cả các split theo lô.


# Cố gắng nạp dataset đã tokenized sẵn từ cache "tokenized_datasets" để tiết kiệm thời gian
try:
    # Thử nạp từ cache nếu đã tokenized trước đó
    tokenized_datasets = load_from_disk("tokenized_datasets") 
    print("Loaded Tokenized Dataset")
except:
    # Nếu chưa có cache, thực thi map → loại bỏ cột thừa → lưu cache
    tokenized_datasets = dataset.map(tokenize_function, batched=True) # Áp dụng hàm tokenize_function cho toàn bộ dataset
    tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer']) # Loại bỏ cột thừa để tiết kiệm bộ nhớ

    # Lưu dataset đã tokenized vào cache "tokenized_datasets" để lần sau dùng lại
    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("Tokenized and Saved Dataset")

Map:   0%|          | 0/118695 [00:00<?, ? examples/s]

Map:   0%|          | 0/14835 [00:00<?, ? examples/s]

Map:   0%|          | 0/14838 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/118695 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14835 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14838 [00:00<?, ? examples/s]

Tokenized and Saved Dataset


In [ ]:
# Hiển thị thông tin tokenized_datasets kiểm tra nhanh khóa/cấu trúc trước khi train
print(tokenized_datasets.keys()) # Hiển thị các split có trong tokenized_datasets
print(tokenized_datasets['train'][0].keys()) # Hiển thị các khóa trong một mẫu của tập train
print(tokenized_datasets['train'][0]['input_ids'][:10]) # Hiển thị 10 token đầu tiên của input_ids
print(tokenized_datasets['train'][0]['labels'][:10]) # Hiển thị 10 token đầu tiên của labels
print(tokenized_datasets) # Hiển thị thông tin dataset

dict_keys(['train', 'test', 'validation'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]
[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}") # In kích thước tập train (số mẫu, số cột)
print(f"Validation: {tokenized_datasets['validation'].shape}") # In kích thước tập validation (số mẫu, số cột)
print(f"Test: {tokenized_datasets['test'].shape}") # In kích thước tập test (số mẫu, số cột)

print(tokenized_datasets) # Hiển thị thông tin tokenized_datasets

Shapes of the datasets:
Training: (118695, 2)
Validation: (14838, 2)
Test: (14835, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


# ==============================
# 7) KIỂM THỬ ZERO-SHOT VỚI MODEL GỐC (BASELINE)
# ==============================

In [ ]:
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor

# Định nghĩa prompt & answer mẫu để test nhanh (người dùng cần thay bằng ví dụ thật)
prompt = "Your input prompt here"  # Ví dụ: Tables + Question (cần thay bằng prompt thực)
answer = "Expected human response here"  # Ví dụ: câu SQL chuẩn tương ứng (đáp án chuẩn)

# Đảm bảo model và input cùng trên một thiết bị (CPU hoặc GPU) nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Di chuyển model sang thiết bị phù hợp (device = CPU hoặc GPU)
original_model.to(device)

# Tokenize đầu vào input và chuyển nó sang cùng thiết bị với model
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# Sinh đầu ra zero-shot từ model gốc (chưa fine-tune)
output = tokenizer.decode( # dùng tokenizer.decode để giải mã token thành text
    original_model.generate( # dùng phương thức generate để sinh text
        inputs["input_ids"], # input_ids của prompt đã tokenized
        max_new_tokens=200, # Giới hạn tối đa 200 token mới sinh
    )[0], # Lấy mảng token đầu tiên trong batch (ở đây batch size=1)
    skip_special_tokens=True # Bỏ qua các token đặc biệt khi giải mã
)

# In kết quả zero-shot baseline
dash_line = '-' * 100 
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}') 
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Your input prompt here
----------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
Expected human response here

----------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Votre prompt here


# ==============================
# 8) HUẤN LUYỆN FULL FINE-TUNE (KAGGLE GPU)
# ==============================

#### 5e-3

Thời gian huấn luyện      = 2 giờ 49 phút 1 giây trên laptop sử dụng notebook Kaggle (PC cá nhân không đủ bộ nhớ CUDA để huấn luyện với tập dữ liệu lớn)

Training Loss   = 0.023100

Validation Loss = 0.013285

---

**Giải thích:**

- **5e-3**: Đây là giá trị learning rate (tốc độ học) được sử dụng khi huấn luyện mô hình. Giá trị này ảnh hưởng đến tốc độ cập nhật trọng số của mô hình trong quá trình học.
- **Thời gian huấn luyện**: Tổng thời gian để hoàn thành quá trình fine-tune mô hình trên GPU của Kaggle. Nếu dùng máy cá nhân (PC) không có đủ bộ nhớ CUDA thì sẽ không thể huấn luyện với tập dữ liệu lớn.
- **Training Loss**: Độ lỗi (loss) trên tập huấn luyện. Giá trị càng nhỏ chứng tỏ mô hình học tốt trên dữ liệu huấn luyện.
- **Validation Loss**: Độ lỗi trên tập kiểm thử (validation). Giá trị này dùng để đánh giá khả năng tổng quát hóa của mô hình trên dữ liệu chưa từng thấy. Nếu validation loss thấp và gần với training loss, mô hình không bị overfit.

**Kết luận:**  
Mô hình đã được huấn luyện với tốc độ học 5e-3, thời gian gần 3 tiếng trên GPU của Kaggle. Kết quả training loss và validation loss đều thấp, chứng tỏ mô hình học tốt và có khả năng tổng quát hóa tốt trên dữ liệu kiểm thử.

In [ ]:
# Cố gắng nạp model đã fine-tune từ "finetuned_model_2_epoch" nếu có
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch") # Nạp model đã fine-tune nếu có
    finetuned_model = finetuned_model.to('cuda') # Chuyển model sang GPU nếu có
    to_train = False # Nếu có model fine-tune, không cần train lại

except:
    # Nếu không có model fine-tune, nạp model gốc để fine-tune
    to_train = True # Nếu không có model fine-tune, sẽ tiến hành fine-tune
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16) # Nạp model gốc với dtype bfloat16
    #finetuned_model = finetuned_model.to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name) # Nạp lại tokenizer từ model nền

In [ ]:
%%time # Đo thời gian chạy cell này trên Kaggle 
import os
os.environ["WANDB_DISABLED"] = "true" # Tắt Weights & Biases để tránh lỗi khi train trên Kaggle

if to_train:
    # Tạo thư mục output duy nhất với timestamp để lưu kết quả train
    output_dir = f'./sql-training-{str(int(time.time()))}'

    # Cấu hình tham số huấn luyện
    training_args = TrainingArguments(
        output_dir=output_dir, # Thư mục lưu kết quả huấn luyện
        learning_rate=5e-3, # Tốc độ học của mô hình (càng lớn càng nhanh nhưng dễ quá khớp)
        num_train_epochs=1, # Số epoch huấn luyện (có thể tăng lên 2,3,... để cải thiện độ chính xác nhưng tốn thời gian hơn)
        per_device_train_batch_size=16, # Dùng batch size 16 (có thể tăng lên nếu GPU đủ lớn, giúp mô hình học tốt hơn)
        per_device_eval_batch_size=16, # Dùng batch size 16 cho đánh giá
        weight_decay=0.01, # Hệ số weight decay để tránh overfitting (thường chọn 0.01)
        logging_steps=50, # Ghi log mỗi 50 bước
        evaluation_strategy='steps', # Đánh giá theo số bước để tránh overfitting, thường chọn 'steps' vì dataset nhỏ
        eval_steps=500, # Đánh giá mỗi 500 bước
    )

    # Tạo Trainer để huấn luyện mô hình
    trainer = Trainer(
        model=finetuned_model, # Mô hình cần huấn luyện (ở đây là model gốc chưa fine-tune)
        args=training_args, # Tham số huấn luyện đã cấu hình
        train_dataset=tokenized_datasets['train'], # Dữ liệu huấn luyện
        eval_dataset=tokenized_datasets['validation'], # Dữ liệu đánh giá (validation)
    )

    trainer.train() # Bắt đầu huấn luyện mô hình
    print("Training completed successfully!") # In thông báo hoàn thành huấn luyện


   # Lưu checkpoint fine-tuned (thư mục mặc định ở working dir)
    finetuned_model.save_pretrained("finetuned_model_2_epoch") 

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.060800,0.042019
1000,0.040200,0.030493
1500,0.036500,0.025905
2000,0.030900,0.022786
2500,0.024600,0.020304
3000,0.025800,0.018751
3500,0.028700,0.017546
4000,0.022300,0.016787
4500,0.023800,0.016067
5000,0.020200,0.015377


Training completed successfully!
CPU times: user 2h 24min 29s, sys: 27min 16s, total: 2h 51min 46s
Wall time: 2h 51min 37s


In [ ]:
# Dù đã fine-tune hay chưa, vẫn lưu model/tokenizer vào thư mục làm việc của Kaggle để tải về
finetuned_model.save_pretrained("/kaggle/working/sql_t5_finetuned") # Lưu model đã fine-tune vào thư mục làm việc của Kaggle
tokenizer.save_pretrained("/kaggle/working/sql_t5_finetuned") # Lưu tokenizer vào thư mục làm việc của Kaggle


('/kaggle/working/sql_t5_finetuned/tokenizer_config.json',
 '/kaggle/working/sql_t5_finetuned/special_tokens_map.json',
 '/kaggle/working/sql_t5_finetuned/spiece.model',
 '/kaggle/working/sql_t5_finetuned/added_tokens.json',
 '/kaggle/working/sql_t5_finetuned/tokenizer.json')

In [ ]:
# Hiển thị model đã fine-tune (hoặc model gốc nếu chưa fine-tune)
print("Model:", finetuned_model) 


Model: T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout

# ==============================
# 9) KIỂM THỬ MÔ HÌNH ĐÃ FINE-TUNE (ZERO-SHOT TRÊN MẪU)
# ==============================

In [ ]:
index = 0 # Thay đổi index để kiểm tra các mẫu khác nhau trong tập test (lấy mẫu đầu tiên để minh họa)
# index = len(dataset['test'])-200

question = dataset['test'][index]['question'] # Lấy câu hỏi từ tập test
context = dataset['test'][index]['context'] # Lấy context từ tập test
answer = dataset['test'][index]['answer'] # Lấy câu trả lời chuẩn từ tập test

# Lại tạo prompt theo cùng "khuôn" như tokenization để mô hình hiểu
# Khuôn mẫu: Tables: <context>  Question: <question>  Answer:
prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

# Đảm bảo model và input cùng trên một thiết bị (CPU hoặc GPU) nếu có
inputs = tokenizer(prompt, return_tensors='pt') # Tokenize đầu vào
inputs = inputs.to('cuda') # Chuyển input sang GPU nếu có

output = tokenizer.decode( # dùng tokenizer.decode để giải mã token thành text
    finetuned_model.generate( # dùng finetuned_model (vừa train hoặc model gốc nếu chưa train)
        inputs["input_ids"], # input_ids của prompt đã tokenized
        max_new_tokens=200, # Giới hạn tối đa 200 token mới sinh
    )[0], # Lấy mảng token đầu tiên trong batch (ở đây batch size=1)
    skip_special_tokens=True # Bỏ qua các token đặc biệt khi giải mã
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)

Question:
On what Date did the Away team essendon play?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT date FROM table_name_11 WHERE away_team = "essendon"

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
SELECT date FROM table_name_11 WHERE away_team = "essendon"


# ==============================
# 10) ĐÁNH GIÁ SƠ BỘ BẰNG ROUGE (SUBSET 25 MẪU)
# ==============================
 Ghi chú: ROUGE đo độ “giống” chuỗi, không phải thước đo chuẩn cho SQL (Exec-Acc), nhưng dùng ở đây như chỉ báo tham khảo nhanh.

In [ ]:
# Đánh giá mô hình định lượng bằng metric ROUGE (subset 25 mẫu do thời gian chạy lâu)
questions = dataset['test'][0:25]['question'] # Lấy 25 câu hỏi từ tập test
contexts = dataset['test'][0:25]['context'] # Lấy 25 context từ tập test
human_baseline_answers = dataset['test'][0:25]['answer'] # Lấy 25 câu trả lời chuẩn từ tập test

original_model_answers = [] # Lưu câu trả lời từ model gốc (baseline)
finetuned_model_answers = [] # Lưu câu trả lời từ model đã fine-tune

for idx, question in enumerate(questions): # Lặp qua từng câu hỏi trong 25 mẫu

    # Tạo prompt đầu vào cho từng mẫu test theo đúng khuôn mẫu đã dùng khi train
    prompt = f"""Tables:
{contexts[idx]}

Question:
{question}

Answer:
"""

    # Tokenize prompt: chuyển prompt thành input_ids để model hiểu (dùng tokenizer đã fine-tune)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda') # Đảm bảo input nằm trên GPU (nếu có)

    # Lấy đáp án chuẩn (human_baseline_text_output) để so sánh với model
    human_baseline_text_output = human_baseline_answers[idx] 

    # Sinh đáp án từ model gốc (chưa fine-tune)
    # GenerationConfig(max_new_tokens=300): giới hạn tối đa 300 token mới sinh ra (tránh sinh quá dài)
    original_model_outputs = original_model.generate(
        input_ids=input_ids, 
        generation_config=GenerationConfig(max_new_tokens=300)
    )
    # Giải mã token thành text, bỏ qua các token đặc biệt (skip_special_tokens=True)
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_answers.append(original_model_text_output) # Lưu kết quả vào list

    # Sinh đáp án từ model đã fine-tune (huấn luyện trên tập dữ liệu SQL)
    finetuned_model_outputs = finetuned_model.generate(
        input_ids=input_ids, 
        generation_config=GenerationConfig(max_new_tokens=300)
    )
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_text_output) # Lưu kết quả vào list

# Ghép 3 loại đáp án (chuẩn, model gốc, model fine-tune) thành từng dòng để tiện so sánh
zipped_summaries = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))

# Tạo DataFrame pandas để hiển thị kết quả so sánh từng mẫu
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_answers', 'original_model_answers', 'finetuned_model_answers'])
# df # (bỏ comment nếu muốn hiển thị bảng kết quả)

Token indices sequence length is longer than the specified maximum sequence length for this model (1115 > 512). Running this sequence through the model will result in indexing errors



Các bước phía dưới sẽ thực hiện đánh giá định lượng mô hình đã fine-tune bằng metric ROUGE trên một tập mẫu nhỏ (25 mẫu) để so sánh chất lượng sinh SQL giữa model gốc và model đã huấn luyện.  
Sau đó, hướng dẫn cách sử dụng mô hình đã fine-tune để sinh truy vấn SQL mới (inference), và đóng gói mô hình thành file zip để tiện tải về hoặc triển khai vào ứng dụng thực tế (FastAPI, Flask, .NET...).

**Tóm tắt các bước:**
1. Đánh giá mô hình bằng ROUGE (so sánh với đáp án chuẩn).
2. Sinh truy vấn SQL mới từ mô hình đã fine-tune (inference).

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=eb239942ed430dd9a7751a018ff5f8b4f9e406835500de47397bb6e7fb145f08
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


Compute ROUGE score for this subset of the data.

In [ ]:
rouge = evaluate.load('rouge') # Nạp metric ROUGE từ thư viện evaluate

original_model_results = rouge.compute( # Tính toán ROUGE cho model gốc
    predictions=original_model_answers, # Dự đoán từ model gốc
    references=human_baseline_answers[0:len(original_model_answers)], # Đáp án chuẩn tương ứng
    use_aggregator=True, # Sử dụng hàm tổng hợp kết quả
    use_stemmer=True, # Sử dụng stemming để cải thiện so khớp
)
print('ORIGINAL MODEL:')
print(original_model_results) # In kết quả ROUGE cho model gốc


finetuned_model_results = rouge.compute( # Tính toán ROUGE cho model đã fine-tune
    predictions=finetuned_model_answers, # Dự đoán từ model đã fine-tune
    references=human_baseline_answers[0:len(finetuned_model_answers)], # Đáp án chuẩn tương ứng
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results) # In kết quả ROUGE cho model đã fine-tune

ORIGINAL MODEL:
{'rouge1': 0.031233998975934457, 'rouge2': 0.005, 'rougeL': 0.03151917519331407, 'rougeLsum': 0.03174603174603174}
FINE-TUNED MODEL:
{'rouge1': 0.9200182907378598, 'rouge2': 0.9027459586080278, 'rougeL': 0.916619247172085, 'rougeLsum': 0.9139777174406729}


# ==============================
# 11) HÀM SUY LUẬN (INFERENCE) VỚI MÔ HÌNH ĐÃ FINE-TUNE
# ==============================

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer # Model và Tokenizer T5 (nạp lại để chắc chắn)
import torch

# Nạp mô hình đã fine-tune và tokenizer từ thư mục lưu trên Kaggle
model_path = "sql_t5_finetuned" # Thư mục ngay dưới /kaggle/working
model = T5ForConditionalGeneration.from_pretrained(model_path).to('cuda')
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Hàm tiện ích để sinh truy vấn SQL từ context và question
def generate_sql(context, question):
    prompt = f"""Tables:
    {context}

    Question:
    {question}

    Answer:
    """
    
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=200,
        )[0],
        skip_special_tokens=True
    )

    return output

# Ví dụ inference với một mẫu từ tập test (phần tử đầu tiên)
index = 0
context = dataset['test'][index]['context']
question = dataset['test'][index]['question']

output = generate_sql(context, question)

print(f"Generated SQL Query:\n{output}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Generated SQL Query:
SELECT date FROM table_name_11 WHERE away_team = "essendon"


# ==============================
# 12) ĐÓNG GÓI MÔ HÌNH ĐỂ TẢI XUỐNG
# ==============================

In [ ]:
# Cài thêm để chạy service nếu cần (không bắt buộc)
!pip install fastapi uvicorn transformers torch


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import shutil # Thư viện shutil để nén file zip

# Nén thư mục model đã fine-tune để tiện download từ Kaggle
shutil.make_archive('/kaggle/working/sql_t5_finetuned', 'zip', '/kaggle/working/sql_t5_finetuned')


'/kaggle/working/sql_t5_finetuned.zip'